<a href="https://colab.research.google.com/github/bruno353/Data_Science_Projects/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/arashnic/book-recommendation-dataset

In [13]:
import pandas as pd

In [14]:
df_ratings = pd.read_csv('Ratings.csv')

In [21]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [15]:
df_ratings.shape

(1149780, 3)

In [16]:
df_books = pd.read_csv('Books.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
df_books.shape

(271360, 8)

In [18]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [23]:
#SELECIONANDO PESSOAS QUE JÁ DERAM REVIEWS EM MAIS DE 200 LIVROS:
df_ratings=df_ratings[df_ratings['Book-Rating']>0]

In [25]:
counts1 = df_ratings['User-ID'].value_counts()
df_ratings= df_ratings[df_ratings['User-ID'].isin(counts1[counts1 > 200].index)]
df_ratings

,User-ID,ISBN,Book-Rating
14854,2276,0020960808,10
14856,2276,0030632366,9
14858,2276,0061030643,8
14860,2276,0061098353,8
14861,2276,0061099155,9
...,...,...,...
1139786,274061,1892213737,10
1139787,274061,189221394X,10
1139788,274061,1892213958,10
1139789,274061,1892213966,10


In [26]:
cdf1=df_ratings[['User-ID','ISBN','Book-Rating']]
cdf=pd.merge(cdf1,df_books,on='ISBN')
cdf

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,2276,0020960808,10,The Natural Superiority of Women,Ashley Montagu,1979,Collier Books,http://images.amazon.com/images/P/0020960808.0...,http://images.amazon.com/images/P/0020960808.0...,http://images.amazon.com/images/P/0020960808.0...
1,2276,0030632366,9,Outrageous Acts and Everyday Rebellions,Gloria Steinem,1987,Smithmark Pub,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...
2,2276,0061030643,8,The Associate,Phillip Margolin,2002,HarperTorch,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...
3,11676,0061030643,7,The Associate,Phillip Margolin,2002,HarperTorch,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...
4,172742,0061030643,7,The Associate,Phillip Margolin,2002,HarperTorch,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...
...,...,...,...,...,...,...,...,...,...,...
62183,274061,1892213737,10,"Cardcaptor Sakura, Number 5",Clamp,2001,Tokyopop,http://images.amazon.com/images/P/1892213737.0...,http://images.amazon.com/images/P/1892213737.0...,http://images.amazon.com/images/P/1892213737.0...
62184,274061,189221394X,10,Clover #2,Clamp,2001,Tokyopop,http://images.amazon.com/images/P/189221394X.0...,http://images.amazon.com/images/P/189221394X.0...,http://images.amazon.com/images/P/189221394X.0...
62185,274061,1892213958,10,Clover #3,Clamp,2002,Tokyopop,http://images.amazon.com/images/P/1892213958.0...,http://images.amazon.com/images/P/1892213958.0...,http://images.amazon.com/images/P/1892213958.0...
62186,274061,1892213966,10,"Clover (Clover, 4)",Clamp,2002,TokyoPop,http://images.amazon.com/images/P/1892213966.0...,http://images.amazon.com/images/P/1892213966.0...,http://images.amazon.com/images/P/1892213966.0...


In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cdf['User_ID']=le.fit_transform(cdf['User-ID'])
cdf['title_id']=le.fit_transform(cdf['Book-Title'])
cdf.drop('User-ID',axis=1,inplace=True)
cdf

,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User_ID,title_id
0,0020960808,10,The Natural Superiority of Women,Ashley Montagu,1979,Collier Books,http://images.amazon.com/images/P/0020960808.0...,http://images.amazon.com/images/P/0020960808.0...,http://images.amazon.com/images/P/0020960808.0...,0,36440
1,0030632366,9,Outrageous Acts and Everyday Rebellions,Gloria Steinem,1987,Smithmark Pub,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...,http://images.amazon.com/images/P/0030632366.0...,0,24193
2,0061030643,8,The Associate,Phillip Margolin,2002,HarperTorch,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,0,31472
3,0061030643,7,The Associate,Phillip Margolin,2002,HarperTorch,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,7,31472
4,0061030643,7,The Associate,Phillip Margolin,2002,HarperTorch,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,http://images.amazon.com/images/P/0061030643.0...,96,31472
...,...,...,...,...,...,...,...,...,...,...,...
62183,1892213737,10,"Cardcaptor Sakura, Number 5",Clamp,2001,Tokyopop,http://images.amazon.com/images/P/1892213737.0...,http://images.amazon.com/images/P/1892213737.0...,http://images.amazon.com/images/P/1892213737.0...,143,6066
62184,189221394X,10,Clover #2,Clamp,2001,Tokyopop,http://images.amazon.com/images/P/189221394X.0...,http://images.amazon.com/images/P/189221394X.0...,http://images.amazon.com/images/P/189221394X.0...,143,7029
62185,1892213958,10,Clover #3,Clamp,2002,Tokyopop,http://images.amazon.com/images/P/1892213958.0...,http://images.amazon.com/images/P/1892213958.0...,http://images.amazon.com/images/P/1892213958.0...,143,7030
62186,1892213966,10,"Clover (Clover, 4)",Clamp,2002,TokyoPop,http://images.amazon.com/images/P/1892213966.0...,http://images.amazon.com/images/P/1892213966.0...,http://images.amazon.com/images/P/1892213966.0...,143,7032


In [36]:
!pip install surprise
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import Dataset
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(cdf[['User_ID','title_id','Book-Rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4779  1.4701  1.5209  1.4674  1.4907  1.4854  0.0195  
MAE (testset)     1.0899  1.0840  1.1198  1.0772  1.0873  1.0916  0.0147  
Fit time          3.59    3.70    3.63    3.70    3.62    3.65    0.04    
Test time         0.22    0.09    0.09    0.10    0.09    0.12    0.05    


{'fit_time': (3.5874342918395996,
  3.6983706951141357,
  3.632429838180542,
  3.695540428161621,
  3.620316982269287),
 'test_mae': array([1.08985706, 1.0839942 , 1.11979285, 1.07718782, 1.0872611 ]),
 'test_rmse': array([1.47785384, 1.47008664, 1.52093171, 1.46737206, 1.49073391]),
 'test_time': (0.21794867515563965,
  0.09154582023620605,
  0.09458494186401367,
  0.09972620010375977,
  0.0896916389465332)}

In [41]:
def user_rec(id):
    user= cdf[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','title_id']].copy()
    user = user.reset_index()
# getting full dataset
    data = Dataset.load_from_df(cdf[['User_ID','title_id','Book-Rating']], reader)
    trainset = data.build_full_trainset()
    svd.fit(trainset)
    user['Estimate_Score'] = user['title_id'].apply(lambda x: svd.predict(id, x).est)
    user = user.drop(['index','title_id'], axis = 1)
    user= user.sort_values('Estimate_Score' , ascending = False)
    counts1 = user['Estimate_Score'].value_counts()
    user = user[user['Estimate_Score'].isin(counts1[counts1 == 1].index)]
    return user.head(10)

In [43]:
pd.set_option('display.max_rows',None) 
Uid=4
print("THE ID OF THE USER: ",Uid)
details=cdf.loc[cdf['User_ID']==Uid]
id=details['User_ID'].iloc[0]
a=user_rec(id)
a.reset_index(inplace=True)
a.drop(['index'],axis=1,inplace=True)
details.reset_index(inplace=True)
details
details.drop(['index','User_ID','title_id','ISBN'],axis=1,inplace=True)
print("\n********************************************USER HAS RATED THESE BOOKs******************************************************")
title_1=list(details['Book-Title'])
rat1=list(details['Book-Rating'])
for i,j in zip(title_1,rat1):
    print(i,'=>',j)
print("\n****************BOOKS RECOMENDATIONS:******************")

display(a)

THE ID OF THE USER:  4

********************************************USER HAS RATED THESE BOOKs******************************************************
Dirk Gently's Holistic Detective Agency => 10
Snow Falling on Cedars => 8
Angela's Ashes: A Memoir => 8
HOW GREEN WAS MY VALLEY => 10
Z for Zachariah => 8
Food for Fifty => 7
The Lion, the Witch and the Wardrobe => 10
How to Cook Everything: Simple Recipes for Great Food => 10
Mutant Message Down Under => 8
All I Know About Animal Behavior I Learned in Loehmann's Dressing Room => 9
Where the Sidewalk Ends: The Poems and Drawings of Shel Silverstein => 10
A Light in the Attic => 10
A Light in the Attic => 10
Stupid White Men ...and Other Sorry Excuses for the State of the Nation! => 10
What To Cook When You Think There's Nothing in the House To Eat : More Than 175 Easy Recipes And Meal Ideas => 7
Cheaper and Better: Homemade Alternatives => 8
One for the Money (Stephanie Plum Novels (Paperback)) => 7
Terms of Adornment : The Ultimate Guide 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Estimate_Score
0,0689304374,The CHURCH MICE &amp; THE MOON (Church Mice &a...,Graham Oakley,1974,Atheneum,9.653164
1,0062505238,Black Elk: The Sacred Ways of the Lakota (Nati...,Wallace Black Elk,1990,Harpercollins,9.652405
2,0747543992,The Glamour Chase,Tom Doyle,1999,Bloomsbury Publishing Plc,9.597739
3,0590630954,Harvey Potters Balloon Farm,Jerdine Nolen,0,Scholastic Inc,9.592615
4,0345333551,Lucid Dreaming,Stephen Laberge,1986,Ballantine Books,9.583585
5,0312112629,Depeche Mode: Some Great Reward,Dave Thompson,1994,St. Martin's Press,9.577013
6,1551666847,Cold Ridge,Carla Neggers,2003,Mira,9.555238
7,0440139880,Immigrant's Daughter,Howard Fast,1987,Dell Publishing Company,9.547257
8,078686382X,Disneys Family Cookbk-OS,Deanna F. Cook,1996,Disney Press,9.512091
9,1559948787,Men are from Mars Women are from Venus,John Gray,2004,HarperAudio,9.504633
